# 

In [2]:
import pandas as pd
import numpy as np
import pyproj
import folium
import os

In [3]:
os.getcwd()

'd:\\05_Send'

In [4]:
df = pd.read_csv(r"d:\05_Send\fulldata_07_24_04_P_일반음식점.csv",
                 encoding='cp949',
                 usecols=['좌표정보x(epsg5174)','좌표정보y(epsg5174)'])

In [5]:
df.head()


,좌표정보x(epsg5174),좌표정보y(epsg5174)
0,262309.010483,511826.091243
1,262199.752428,511689.812677
2,262166.278134,512435.131903
3,262309.010483,511826.091243
4,262290.842004,512049.482131


In [6]:
len(df)


2242009

In [7]:
df


,좌표정보x(epsg5174),좌표정보y(epsg5174)
0,262309.010483,511826.091243
1,262199.752428,511689.812677
2,262166.278134,512435.131903
3,262309.010483,511826.091243
4,262290.842004,512049.482131
...,...,...
2242004,183338.073439,178369.322568
2242005,192085.702260,182111.703093
2242006,190363.418899,181905.028328
2242007,188322.201767,183130.570776


In [8]:
df = df.dropna()
df.index=range(len(df))
df.tail()

,좌표정보x(epsg5174),좌표정보y(epsg5174)
2114305,183338.073439,178369.322568
2114306,192085.702260,182111.703093
2114307,190363.418899,181905.028328
2114308,188322.201767,183130.570776
2114309,188873.020499,184152.828430


In [9]:
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]


def project_array_modern(coord, p1_type, p2_type):
    """
    좌표계 변환 함수 (Coordinate Transformation Function)
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) "epsg:5179"
    - p2_type: 출력 좌표계 정보 ex) "epsg:4326"
    """
    
    # 1. Address FutureWarning: '+init=<authority>:<code>' syntax is deprecated
    #    Use pyproj.Transformer.from_crs, passing the EPSG strings directly.
    #    This avoids initializing pyproj.Proj manually.
    transformer = pyproj.Transformer.from_crs(
        p1_type, 
        p2_type, 
        # always_xy=True ensures the input/output order is always (longitude, latitude) 
        # or (easting, northing) which is usually (x, y). 
        # This helps manage axis order changes.
        always_xy=True 
    )

    # 2. Address FutureWarning: This function is deprecated (pyproj.transform)
    #    Use the modern transformer.transform() method instead.
    fx, fy = transformer.transform(coord[:, 0], coord[:, 1])

    return np.dstack([fx, fy])[0]

In [10]:
coord = np.array(df)
coord


array([[262309.01048315, 511826.09124281],
       [262199.75242819, 511689.81267735],
       [262166.27813424, 512435.13190268],
       ...,
       [190363.41889933, 181905.02832814],
       [188322.20176694, 183130.57077595],
       [188873.02049868, 184152.82843045]], shape=(2114310, 2))

In [11]:
p1_type = "epsg:5174"
p2_type = "epsg:4326"

result = project_array_modern(coord, p1_type, p2_type)
result


array([[127.71121447,  38.10714574],
       [127.70995697,  38.10592561],
       [127.70964015,  38.11264201],
       ...,
       [126.89507187,  35.13621639],
       [126.87265733,  35.14724115],
       [126.87868814,  35.15646158]], shape=(2114310, 2))

In [14]:
# df['경도'] = result[:, 0]
# df['위도'] = result[:, 1]
df.loc[:, '경도'] = result[:, 0] # Assign E-W coordinate (경도)
df.loc[:, '위도'] = result[:, 1] # Assign N-S coordinate (위도)
df.tail()



,좌표정보x(epsg5174),좌표정보y(epsg5174),경도,위도
2114305,183338.073439,178369.322568,126.818058,35.104255
2114306,192085.702260,182111.703093,126.913968,35.138094
2114307,190363.418899,181905.028328,126.895072,35.136216
2114308,188322.201767,183130.570776,126.872657,35.147241
2114309,188873.020499,184152.828430,126.878688,35.156462


In [13]:
# 데이터 100개 랜덤 추출
sample = df.sample(n=100)

# 지도 중심 좌표 설정
lat_c, lon_c = 37.53165351203043, 126.9974246490573

# Folium 지도 객체 생성
m = folium.Map(location=[lat_c, lon_c], zoom_start=6)

# 마커 생성
for _, row in sample.iterrows():
    lat, lon = row['위도'], row['경도']
    folium.Marker(location=[lat, lon]).add_to(m)
    
m